# Import library

In [1]:
# get data
from binance.spot import Spot 
from constants import price_cols, PERIOD, SYMBOL, DURATION_EACH_DAY, DATA_POINT_ONE_DAY, NUM_OF_DAYS

# train data
import pandas as pd
from datetime import datetime

# enviroment
import os
from dotenv import load_dotenv
load_dotenv("../../env/app.env")

True

# Setup clients

In [2]:
BINANCE_API_KEY = os.environ.get("BINANCE_API_KEY")
BINANCE_API_SECRET = os.environ.get("BINANCE_API_SECRET")
client = Spot(key=BINANCE_API_KEY, secret=BINANCE_API_SECRET)

# Fetch day period data

### Get data points of symbols 

In [3]:
exchanges = client.exchange_info()
symbols = list(map(lambda item: item.get('symbol'), exchanges.get('symbols')))

In [4]:
print(f"{DATA_POINT_ONE_DAY * NUM_OF_DAYS} data points")

172800 data points


In [5]:
today = datetime.utcnow().replace(hour=23, minute=0, second=0, microsecond=0)
today_timestamp = int(today.timestamp() * 1000)

start_timestamp = today_timestamp - NUM_OF_DAYS * DURATION_EACH_DAY
end_timestamp = today_timestamp

try:
    for timestamp in range(start_timestamp, end_timestamp, DURATION_EACH_DAY):
        data = client.klines(SYMBOL, PERIOD, limit=1000, startTime=timestamp, endTime=timestamp + DURATION_EACH_DAY)
        if start_timestamp == timestamp:
            df = pd.DataFrame(data, columns=price_cols)
        else:
            df = pd.concat([df, pd.DataFrame(data, columns=price_cols)], axis=0)

    df.drop("Unused field, ignore", axis=1, inplace=True)
    df.to_csv(f"../../datastore/price/{SYMBOL}_{PERIOD}.csv", index=False)
except:
    print(f"Error at {SYMBOL}")